In [9]:
from flask import Flask , jsonify , render_template , request

In [10]:
import mysql.connector

In [11]:
app = Flask(__name__)

In [12]:
def create_connection():
    return mysql.connector.connect(
        host='food', 
        user = 'root',
        password = 'root',
        database = 'zomato'
    )

In [13]:
#Home route
@app.route('/')
def home():
    return render_template('home.html')

In [14]:
# Place an order (POST)

@app.route('/place_order',methods=['POST'])
def place_order():
    cursor = None
    connection = None
    try:
        order_data = request.get_json() # Get the JSON data sent by the frontend

        cart = order_data['cart'] # Array of items
        address = order_data['address'] # delivery address

        if not cart or not address:
            return jsonify({"error":"Cart is missing or address is missing"})
        
        # Enter each item in the cart into the database 
        connection = create_connection()
        cursor = connection.cursor()

        for item in cart:
            item_name = item['name']
            quantity = item['quantity']

            cursor.execute("SELECT item_id FROM items WHERE item_name = %s",(item_name))
            result = cursor.fetchone()

            if not result:
                return jsonify({"error":f"Item {item_name} not found."}),400
            
            item_id = result[0]  # Get the item_id from the query list
            user_id = 0  # Assume

            # SQL query to insert the order
            sql = """
                INSERT INTO orders (user_id , item_id , quantity , delivery_address)
                VALUES(%s , %s , %s , %s)
                """
            cursor.execute(sql , (user_id, item_id , quantity, address))

        connection.commit()
        return jsonify({"message" : "Order placed successfully"}) 
    except Exception as e:
        return jsonify({"error":str(e)}), 500

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

In [15]:
# Get all orders
@app.route('/orders', methods=['GET'])
def get_orders():
    cursor = None
    connection = None
    try :
        connection = create_connection()
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM orders")
        orders = cursor.fetchall()

        return jsonify({"orders":orders}) , 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    finally :
        if cursor:
            cursor.close()
        if connection:
            connection.close()
            
                

In [16]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1